In [3]:
!pip install torchinfo

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Tuple, Dict
import torch
from torch.utils.data import Dataset, DataLoader
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from torchsummary import summary
from torchinfo import summary
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import wandb
import torch.nn.functional as F
import hashlib
from typing import Dict, Tuple
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Configuration Dictionary
config = {
    'batch_size': 64,
    'lr': 0.001,
    'epochs': 10,
    'input_dim': 9,
    'num_classes': 7,
    'hidden_dim': 2048,
    'num_blocks': 6,
    'checkpoint_dir': "/content/drive/MyDrive/IDL/Checkpoint",
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

In [6]:
# Define category mapping
CATEGORIES = {
    'Blueball': 0,
    'Box': 1,
    'Pencilcase': 2,
    'Pinkball': 3,
    'StuffedAnimal': 4,
    'Tennis': 5,
    'Waterbottle': 6,
}

In [7]:
# Path to the folder containing the dataset files
folder_path = "/content/drive/MyDrive/IDL/IDL_Data"

In [8]:
# Stats trackers
total_count = 0
kept_count = 0
valid_file_count = 0
skipped_due_to_missing_waypoints = 0

In [9]:
WAYPOINTS = [
    (30, -30), (30, 30), (15, -30), (15, 30),
    (0, -30), (0, 30), (-15, -30), (-15, 30),
    (-30, -30), (-30, 30), (-30, -30), (30, -30),
    (30, 30), (-30, 30)
]


In [10]:
# Step 1: Load and label dataset
def load_and_label_file(file_path, file_name):
    global total_count
    category = next((key for key in CATEGORIES if key in file_name), None)
    if category is None:
        return pd.DataFrame()

    data = []
    with open(file_path, "r") as f:
        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 10:
                try:
                    timestamp = parts[0]
                    microsec = int(parts[1])
                    x = float(parts[2])
                    y = float(parts[3])
                    x_target = float(parts[4])
                    y_target = float(parts[5])
                    pwm1 = int(parts[6])
                    pwm2 = int(parts[7])
                    pwm3 = int(parts[8])
                    pwm4 = int(parts[9])
                    total_count += 1

                    data.append([
                        timestamp, microsec, x, y, x_target, y_target,
                        pwm1, pwm2, pwm3, pwm4, category, CATEGORIES[category]
                    ])
                except ValueError:
                    continue

    return pd.DataFrame(data, columns=[
        "timestamp", "microseconds", "x", "y", "x_target", "y_target",
        "pwm1", "pwm2", "pwm3", "pwm4", "category", "label"
    ])

In [11]:
# Step 2: Assign sequential waypoint numbers
def assign_sequential_waypoints(df, tol=1.0):
    df = df.reset_index(drop=True)
    wp_index = 0
    assigned_wp = []

    for i in range(len(df)):
        x_t, y_t = df.loc[i, "x_target"], df.loc[i, "y_target"]
        current_expected = WAYPOINTS[wp_index]

        if np.isclose(x_t, current_expected[0], atol=tol) and np.isclose(y_t, current_expected[1], atol=tol):
            assigned_wp.append(wp_index)
        else:
            if wp_index + 1 < len(WAYPOINTS):
                next_expected = WAYPOINTS[wp_index + 1]
                if np.isclose(x_t, next_expected[0], atol=tol) and np.isclose(y_t, next_expected[1], atol=tol):
                    wp_index += 1
                    assigned_wp.append(wp_index)
                else:
                    assigned_wp.append(wp_index)
            else:
                assigned_wp.append(wp_index)

    df["waypoint_number"] = assigned_wp
    return df

In [12]:
# Step 3: Filter out rows where y <= 0
def filter_by_y(df):
    global kept_count
    filtered = df[df["y"] > 0].reset_index(drop=True)
    kept_count += len(filtered)
    return filtered

In [13]:
def process_file(file_path, file_name):
    global valid_file_count

    # Step 1: Load and label
    df = load_and_label_file(file_path, file_name)
    if df.empty:
        return pd.DataFrame()

    # Step 2: Assign waypoint numbers
    df = assign_sequential_waypoints(df)

    # 📌 Show how many waypoints existed before filtering
    waypoint_count_before = df["waypoint_number"].nunique()
    print(f"\n📌 {file_name} → {waypoint_count_before} waypoints BEFORE filtering")

    # Step 3: Filter out rows where y ≤ 0
    df = filter_by_y(df)

    # 📌 Show how many remain after filtering
    waypoint_count_after = df["waypoint_number"].nunique()
    print(f"📌 {file_name} → {waypoint_count_after} waypoints AFTER filtering")

    # Count as valid if any data was kept
    if not df.empty:
        valid_file_count += 1

    return df

In [14]:
# Step 5: Process all .txt files
all_data = pd.DataFrame()

for file_name in os.listdir(folder_path):
    if not file_name.endswith(".txt") or file_name.startswith("."):
        continue
    file_path = os.path.join(folder_path, file_name)
    df = process_file(file_path, file_name)
    if not df.empty:
        all_data = pd.concat([all_data, df], ignore_index=True)

# Step 6: Summary
print("\n📄 Summary:")
print(f"Total files scanned: {len([f for f in os.listdir(folder_path) if f.endswith('.txt')])}")
print(f"✅ Files with 14 valid waypoints: {valid_file_count}")
print(f"⚠️ Skipped due to missing waypoints: {skipped_due_to_missing_waypoints}")
print(f"📊 Data points before filtering: {total_count}")
print(f"✅ Data points after filtering: {kept_count}")
print(f"🚫 Dropped data points: {total_count - kept_count}")


📌 Pinkball2.txt → 14 waypoints BEFORE filtering
📌 Pinkball2.txt → 13 waypoints AFTER filtering

📌 Pinkball6.txt → 14 waypoints BEFORE filtering
📌 Pinkball6.txt → 13 waypoints AFTER filtering

📌 Blueball6.txt → 14 waypoints BEFORE filtering
📌 Blueball6.txt → 13 waypoints AFTER filtering

📌 Waterbottle2.txt → 14 waypoints BEFORE filtering
📌 Waterbottle2.txt → 13 waypoints AFTER filtering

📌 Waterbottle4.txt → 14 waypoints BEFORE filtering
📌 Waterbottle4.txt → 13 waypoints AFTER filtering

📌 Blueball10.txt → 14 waypoints BEFORE filtering
📌 Blueball10.txt → 12 waypoints AFTER filtering

📌 StuffedAnimal7.txt → 14 waypoints BEFORE filtering
📌 StuffedAnimal7.txt → 12 waypoints AFTER filtering

📌 Pinkball5.txt → 14 waypoints BEFORE filtering
📌 Pinkball5.txt → 13 waypoints AFTER filtering

📌 Waterbottle10.txt → 14 waypoints BEFORE filtering
📌 Waterbottle10.txt → 12 waypoints AFTER filtering

📌 Tennis7.txt → 14 waypoints BEFORE filtering
📌 Tennis7.txt → 12 waypoints AFTER filtering

📌 Waterbott

In [15]:
print(all_data.head(1350))

                timestamp  microseconds     x      y  x_target  y_target  \
0     2025-02-28 14:25:35     328046152 -1.27   1.84      30.0     -30.0   
1     2025-02-28 14:25:35     328099628 -1.27   1.84      30.0     -30.0   
2     2025-02-28 14:25:35            68 -1.27   1.84      30.0     -30.0   
3     2025-02-28 14:25:35     328046152 -1.27   1.84      30.0     -30.0   
4     2025-02-28 14:25:35     328099628 -1.27   1.84      30.0     -30.0   
...                   ...           ...   ...    ...       ...       ...   
1345  2025-02-28 14:26:40      64985356  0.64  27.98       0.0      30.0   
1346  2025-02-28 14:26:40      65007184  0.64  27.98       0.0      30.0   
1347  2025-02-28 14:26:40      65029016  0.64  27.98       0.0      30.0   
1348  2025-02-28 14:26:40      65050844  0.64  27.98       0.0      30.0   
1349  2025-02-28 14:26:41      65072664  0.64  27.98       0.0      30.0   

      pwm1  pwm2  pwm3  pwm4  category  label  waypoint_number  
0       80     0    80

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pandas as pd
import numpy as np

In [17]:
class ObjectSensorDataset(Dataset):
    def __init__(self, df):
        features = df[["x", "y", "x_target", "y_target", "pwm1", "pwm2", "pwm3", "pwm4", "microseconds"]].values
        labels = df["label"].values

        self.X = torch.tensor(features, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [18]:
# Normalize features before split
features_to_scale = ["x", "y", "x_target", "y_target", "pwm1", "pwm2", "pwm3", "pwm4", "microseconds"]
scaler = StandardScaler()
all_data[features_to_scale] = scaler.fit_transform(all_data[features_to_scale])

# 🧪 Split into train/val/test with stratified sampling
train_df, temp_df = train_test_split(
    all_data, test_size=0.3, stratify=all_data["label"], random_state=42
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, stratify=temp_df["label"], random_state=42
)

# 📦 Create datasets
train_dataset = ObjectSensorDataset(train_df)
val_dataset = ObjectSensorDataset(val_df)
test_dataset = ObjectSensorDataset(test_df)

# 📦 Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 🧪 Example usage with tqdm (dummy training loop)
for X_batch, y_batch in tqdm(train_loader, desc="🔁 Training"):
    # Simulate training step
    pass

for X_batch, y_batch in tqdm(val_loader, desc="🔍 Validating"):
    # Simulate validation step
    pass

for X_batch, y_batch in tqdm(test_loader, desc="🧪 Testing"):
    # Simulate test step
    pass

# 📊 Summary of dataset sizes
print("\n📊 Dataset Sizes:")
print(f"🧠 Training set: {len(train_dataset)} samples")
print(f"🧪 Validation set: {len(val_dataset)} samples")
print(f"🧾 Test set: {len(test_dataset)} samples")

🧪 Testing: 100%|██████████| 630/630 [00:00<00:00, 1522.84it/s]


📊 Dataset Sizes:
🧠 Training set: 188104 samples
🧪 Validation set: 40308 samples
🧾 Test set: 40309 samples


In [19]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, dim):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.ReLU(inplace=True),
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x
        out = self.block(x)
        out += identity
        return self.relu(out)


class ResidualMLPClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=2048, num_blocks=6):
        super(ResidualMLPClassifier, self).__init__()
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True)
        )

        self.res_blocks = nn.Sequential(*[ResidualBlock(hidden_dim) for _ in range(num_blocks)])

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.input_layer(x)
        x = self.res_blocks(x)
        feats = x
        out = self.classifier(x)
        return {"feats": feats, "out": out}

model = ResidualMLPClassifier(input_dim=9, num_classes=7, hidden_dim = config['hidden_dim'], num_blocks = config['num_blocks']).to(config['device'])
summary(model, input_size=(64, 9))  # for batch size 64


Layer (type:depth-idx)                   Output Shape              Param #
ResidualMLPClassifier                    [64, 7]                   --
├─Sequential: 1-1                        [64, 2048]                --
│    └─Linear: 2-1                       [64, 2048]                20,480
│    └─BatchNorm1d: 2-2                  [64, 2048]                4,096
│    └─ReLU: 2-3                         [64, 2048]                --
├─Sequential: 1-2                        [64, 2048]                --
│    └─ResidualBlock: 2-4                [64, 2048]                --
│    │    └─Sequential: 3-1              [64, 2048]                8,400,896
│    │    └─ReLU: 3-2                    [64, 2048]                --
│    └─ResidualBlock: 2-5                [64, 2048]                --
│    │    └─Sequential: 3-3              [64, 2048]                8,400,896
│    │    └─ReLU: 3-4                    [64, 2048]                --
│    └─ResidualBlock: 2-6                [64, 2048]             

In [20]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [21]:
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [22]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    loss_m = AverageMeter()
    acc_m = AverageMeter()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        x, y = data
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        loss = criterion(outputs['out'], y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        acc = accuracy(outputs['out'], y)[0].item()
        loss_m.update(loss.item())
        acc_m.update(acc)

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(loss_m.avg)),
            acc="{:.04f}%".format(float(acc_m.avg)),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr']))
        )
        batch_bar.update()

        del x, y, outputs, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    return loss_m.avg, acc_m.avg

In [23]:
@torch.no_grad()
def validate_model(model, val_loader, criterion, class_names, device):
    model.eval()
    loss_m = AverageMeter()
    acc_m = AverageMeter()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    all_preds = []
    all_targets = []

    for i, data in enumerate(val_loader):
        x, y = data
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        loss = criterion(outputs['out'], y)

        acc = accuracy(outputs['out'], y)[0].item()

        _, predicted = torch.max(outputs['out'], 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(y.cpu().numpy())

        loss_m.update(loss.item())
        acc_m.update(acc)

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(loss_m.avg)),
            acc="{:.04f}%".format(float(acc_m.avg))
        )
        batch_bar.update()

        del x, y, outputs, loss
        torch.cuda.empty_cache()

    batch_bar.close()

    if class_names:
        print("\nPer-class Validation Accuracy:")
        per_class_acc = {}
        for i, class_name in enumerate(class_names):
            class_mask = (np.array(all_targets) == i)
            if np.sum(class_mask) > 0:
                class_correct = np.sum((np.array(all_preds)[class_mask] == i))
                class_total = np.sum(class_mask)
                acc_percent = 100 * class_correct / class_total
                print(f"  {class_name}: {acc_percent:.4f}% ({class_correct}/{class_total})")
                per_class_acc[f"val_acc_{class_name}"] = acc_percent

    return loss_m.avg, acc_m.avg

In [24]:
def save_model(model, optimizer, scheduler, metrics, epoch, path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'metrics': metrics
    }, path)

In [25]:
# Define CrossEntropyLoss as the criterion
criterion = nn.CrossEntropyLoss(
    label_smoothing=0.1
)

# Initialize optimizer with AdamW
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config['lr'],
    weight_decay=1e-5
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=True
)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [26]:
import wandb

# Intialize wandb
wandb.login(key="78d5988d9f05a421bc74d044c3cd9afc3b918020") # API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: donggul (donggul-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [27]:
run = wandb.init(
    name = "15run", ## Wandb creates random run names if you skip this field
    reinit = False, ### Allows reinitalizing runs when you re-run this cell
    #id = "", ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "object_classification", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

In [35]:
# Training Loop
best_val_loss = float('inf')
best_val_acc = 0
class_names = list(CATEGORIES.keys())

for epoch in range(config['epochs']):
    print(f"\nEpoch {epoch + 1}/{config['epochs']}")

    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, config['device'])
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

    val_loss, val_acc = validate_model(model, val_loader, criterion, class_names, config['device'])
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

    scheduler.step(val_loss)
    curr_lr = optimizer.param_groups[0]['lr']

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_val_acc = val_acc
        best_model_path = os.path.join(config['checkpoint_dir'], 'best_model.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc,
        }, best_model_path)
        wandb.save(best_model_path)
        print(f"Saved best model with validation loss: {best_val_loss:.4f} and accuracy: {best_val_acc:.2f}%")

    last_model_path = os.path.join(config['checkpoint_dir'], f'model_epoch_{epoch+1}.pth')
    torch.save(model.state_dict(), last_model_path)
    wandb.save(last_model_path)
    print(f"Saved model for epoch {epoch+1}")

    wandb.log({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'learning_rate': curr_lr
    }, step=epoch)

    print(f"End of Epoch {epoch+1}/{config['epochs']}")

print(f"\nTraining complete! Best validation accuracy: {best_val_acc:.2f}%")


Epoch 1/10


Train Loss: 1.1262, Train Accuracy: 64.91%



Per-class Validation Accuracy:
  Blueball: 63.9127% (3721/5822)
  Box: 59.3361% (3289/5543)
  Pencilcase: 59.1996% (3195/5397)
  Pinkball: 52.7701% (3067/5812)
  StuffedAnimal: 60.8573% (3677/6042)
  Tennis: 64.1971% (3830/5966)
  Waterbottle: 57.0730% (3268/5726)
Validation Loss: 1.2424, Validation Accuracy: 59.66%
Saved best model with validation loss: 1.2424 and accuracy: 59.66%
Saved model for epoch 1
End of Epoch 1/10

Epoch 2/10


Train:   4%|▍         | 124/2940 [00:06<02:14, 20.98it/s, acc=65.5368%, loss=1.0994, lr=0.001000]wandb: WARNING Tried to log to step 0 that is less than the current step 9. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


Train Loss: 1.0997, Train Accuracy: 66.23%


...

Train Loss: 0.9067, Train Accuracy: 76.50%



Per-class Validation Accuracy:
  Blueball: 49.0210% (2854/5822)
  Box: 44.3442% (2458/5543)
  Pencilcase: 47.6932% (2574/5397)
  Pinkball: 41.3283% (2402/5812)
  StuffedAnimal: 48.0636% (2904/6042)
  Tennis: 55.7828% (3328/5966)
  Waterbottle: 48.1663% (2758/5726)
Validation Loss: 1.7840, Validation Accuracy: 47.82%
Saved model for epoch 9
End of Epoch 9/10

Epoch 10/10


Train:   3%|▎         | 86/2940 [00:04<02:40, 17.74it/s, acc=77.6401%, loss=0.8855, lr=0.000250]wandb: WARNING Tried to log to step 8 that is less than the current step 9. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


Train Loss: 0.8987, Train Accuracy: 76.86%



Per-class Validation Accuracy:
  Blueball: 71.7108% (4175/5822)
  Box: 69.0240% (3826/5543)
  Pencilcase: 67.9452% (3667/5397)
  Pinkball: 71.8858% (4178/5812)
  StuffedAnimal: 72.4429% (4377/6042)
  Tennis: 71.7901% (4283/5966)
  Waterbottle: 69.5599% (3983/5726)
Validation Loss: 1.0556, Validation Accuracy: 70.68%
Saved model for epoch 10
End of Epoch 10/10

Training complete! Best validation accuracy: 68.88%


In [36]:
@torch.no_grad()
def test_model(model, test_loader, criterion, class_names, device, checkpoint_dir=None):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []
    all_probs = []
    class_correct = {class_name: 0 for class_name in class_names}
    class_total = {class_name: 0 for class_name in class_names}
    test_bar = tqdm(test_loader, desc="Testing", unit="batch", ncols=100)

    for data in test_bar:
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        outputs_for_loss = outputs['out'] if isinstance(outputs, dict) and 'out' in outputs else outputs
        loss = criterion(outputs_for_loss, targets)
        test_loss += loss.item() * inputs.size(0)

        probs = torch.nn.functional.softmax(outputs_for_loss, dim=1)
        _, predicted = torch.max(outputs_for_loss, 1)

        total += targets.size(0)
        correct += (predicted == targets).sum().item()

        for i in range(targets.size(0)):
            label = targets[i].item()
            pred = predicted[i].item()
            class_name = class_names[label]
            class_total[class_name] += 1
            if pred == label:
                class_correct[class_name] += 1

        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

        test_bar.set_postfix({
            'loss': f"{test_loss/total:.4f}",
            'acc': f"{100.0*correct/total:.2f}%"
        })

    test_loss /= len(test_loader.dataset)
    test_acc = correct / total

    class_accuracy = {name: class_correct[name]/class_total[name] if class_total[name] > 0 else 0 for name in class_names}

    print("\n" + "="*50)
    print("TEST RESULTS")
    print("="*50)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f} ({correct}/{total})")
    print("\nPer-Class Accuracy:")
    for class_name in class_names:
        print(f" {class_name}: {class_accuracy[class_name]:.4f} ({class_correct[class_name]}/{class_total[class_name]})")

    return {
        'test_loss': test_loss,
        'test_accuracy': test_acc,
        'class_accuracy': class_accuracy,
        'predictions': all_preds,
        'targets': all_targets,
        'probabilities': all_probs
    }

In [37]:
best_model_path = f"{config['checkpoint_dir']}/best_model.pth"
if os.path.exists(best_model_path):
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded best model from epoch {checkpoint.get('epoch', 'unknown')}")

Loaded best model from epoch 2


In [38]:
# Evaluate on test set
test_results = test_model(
    model=model,
    test_loader=test_loader,
    criterion=criterion,
    class_names=class_names,
    device=device,
    checkpoint_dir=config['checkpoint_dir']
)

Testing: 100%|███████████████████████| 630/630 [00:04<00:00, 138.35batch/s, loss=1.0455, acc=68.74%]


TEST RESULTS
Test Loss: 1.0455
Test Accuracy: 0.6874 (27708/40309)

Per-Class Accuracy:
 Blueball: 0.6846 (3986/5822)
 Box: 0.6533 (3621/5543)
 Pencilcase: 0.6717 (3625/5397)
 Pinkball: 0.6999 (4068/5812)
 StuffedAnimal: 0.6925 (4184/6042)
 Tennis: 0.7295 (4352/5966)
 Waterbottle: 0.6761 (3872/5727)


In [ ]:
run.finish()